In [19]:
import librosa
import torch
import numpy as np
import os

# Definieren der Variablen
source_dirs = ["data/knocks", "data/knocks_with_noise", "data/noises"]

# Durchlaufen des Ordners und Vorverarbeiten der Audiodateien
data = []
labels = []
for source_dir in source_dirs:
    for filename in os.listdir(source_dir):
        filepath = os.path.join(source_dir, filename)
        
        # Ignorieren von Verzeichnissen
        if os.path.isdir(filepath):
            continue
        
        # Audio laden
        y, sr = librosa.load(filepath)

        # Berechnen von MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr)

        print(mfcc.shape, len(mfcc), len(mfcc[0]))

        # Hinzufügen einer Kanaldimension
        mfcc = np.expand_dims(mfcc, axis=0)
        print(mfcc.shape, len(mfcc), len(mfcc[0]))


        data.append(mfcc)
        labels.append(1 if filename.startswith("knock") else 0)


from torch.utils.data import Dataset

class AudioDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, self.labels[idx]
    
from torchvision import transforms

# Definieren der Transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
    
dataset = AudioDataset(data, labels, transform=transform)
# Speichern des Datasets
torch.save(dataset, 'dataset.pth')

(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87) 20 87
(1, 20, 87) 1 20
(20, 87)

In [20]:
from torchvision import transforms

dataset = torch.load('dataset.pth')
print(type(dataset), len(dataset))
print(type(dataset.__getitem__(0)))
print(dataset.__getitem__(0))



<class '__main__.AudioDataset'> 274
<class 'tuple'>
(tensor([[[-1.2745e+03,  2.3464e+02, -2.7337e+01,  ...,  4.8698e+00,
          -1.0860e+00,  1.0392e+01]],

        [[-1.2037e+03,  2.3812e+02, -2.9034e+01,  ...,  1.4742e+00,
           2.0559e+00,  8.2397e+00]],

        [[-1.1983e+03,  2.4000e+02, -3.1538e+01,  ..., -5.7936e-01,
          -2.9374e+00,  6.7674e+00]],

        ...,

        [[-1.1934e+03,  2.5568e+02, -1.3737e+01,  ...,  6.0407e-01,
          -9.4601e+00,  9.7316e+00]],

        [[-1.2061e+03,  2.4852e+02, -7.8982e+00,  ...,  4.9105e+00,
          -9.9413e+00,  2.6640e+00]],

        [[-1.2610e+03,  2.4565e+02, -8.5355e+00,  ..., -9.6243e-01,
          -1.0896e+01,  8.8208e+00]]]), 1)


# Version for V7

In [1]:
import os
import shutil
from glob import glob
import random

# Seed for reproducibility
random.seed(42)

# Source directories
source_dirs = ["data/knocks", "data/knocks_with_noise", "data/noises", "data/new_knocks"]

# Destination directories
dest_dirs = {
    "train": "data/trainv2",
    "test": "data/testv2",
    "validation": "data/validationv2"
}

# Collect all sound files
sound_files = []
for source_dir in source_dirs:
    sound_files.extend(glob(os.path.join(source_dir, "*.wav")))

# Shuffle the sound files
random.shuffle(sound_files)

# Split the data
num_test = 20
num_remaining = len(sound_files) - num_test
num_train = int(num_remaining * 0.8)
num_validation = num_remaining - num_train

# Ensure destination directories exist
for dir_path in dest_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# Function to copy files to destination
def copy_files(files, destination):
    for file in files:
        shutil.copy(file, destination)

# Copy files to their respective directories
copy_files(sound_files[:num_test], dest_dirs["test"])
copy_files(sound_files[num_test:num_test + num_train], dest_dirs["train"])
copy_files(sound_files[num_test + num_train:], dest_dirs["validation"])

print(f"Files distributed: {num_test} test, {num_train} train, {num_validation} validation.")

Files distributed: 20 test, 1699 train, 425 validation.


In [2]:
import os

def rename_files_in_directory(directory, start_index=1):
    """
    Renames files in the specified directory that do not contain an underscore in their name.
    Files are renamed to 'knock_new_x', where x is a sequential number starting from start_index.
    Returns the next start index after renaming files in this directory.
    """
    # Get a list of file names in the directory
    files = os.listdir(directory)
    current_index = start_index
    
    for file in files:
        # Check if the file name does not contain an underscore
        if '_' not in file:
            # Define the new file name
            new_file_name = f'knock_new_{current_index}.wav'  # Assuming the files are .wav, change the extension if necessary
            # Rename the file
            os.rename(os.path.join(directory, file), os.path.join(directory, new_file_name))
            # Increment the index for the next file
            current_index += 1
            
    # Return the next start index
    return current_index

# Directories to process
directories = ['data/trainv2', 'data/testv2', 'data/validationv2']

# Start index for renaming
start_index = 1

# Iterate through each directory and rename files
for directory in directories:
    start_index = rename_files_in_directory(directory, start_index)

print("Finished renaming files.")

Finished renaming files.
